# Core Model Guide: TwoDimensionalHydrodynamicModel

This notebook provides a guide to the `TwoDimensionalHydrodynamicModel`, the most advanced hydraulic model in the CHS SDK. It solves the 2D Shallow Water Equations on an unstructured triangular mesh, making it suitable for detailed simulations of flood inundation, estuary dynamics, and other two-dimensional flow problems.

## 1. Theoretical Background

The model is based on the 2D Saint-Venant equations (also known as the Shallow Water Equations), which conserve both mass and momentum in two dimensions (x and y). The solver uses a modern, robust **Finite Volume** method with an **HLLC Approximate Riemann Solver**. This allows it to handle complex situations like wetting and drying, and shocks (rapid changes in water level), making it very powerful.

A key feature is its use of **unstructured meshes**, which allows it to represent complex geometries like winding rivers and floodplains much more efficiently than a simple rectangular grid.

## 2. API and Parameters

The model is initialized by pointing it to a mesh file.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.two_dimensional_hydrodynamic_model import TwoDimensionalHydrodynamicModel

## 3. Code Example: Radial Dam Break (Column Collapse)

A classic validation case for 2D shallow water models is the "column collapse" or "radial dam break". In this scenario, a cylinder of water in an otherwise dry, flat domain is instantaneously released. The water column collapses under gravity, sending out a circular wave.

We will use the simple `square.msh` file we created for the domain.

In [ ]:
# 1. Initialize the Model
mesh_file = '../data/meshes/square.msh'
model = TwoDimensionalHydrodynamicModel(
    mesh_file=mesh_file,
    manning_n=0.02,
    initial_h=0.1, # A small base water depth to keep cells wet
    cfl=0.45
)

# 2. Set Initial Conditions for the Column Collapse
dm = model.data_manager
mesh = model.mesh

# Find cell centers and their distance from the domain center (0.5, 0.5)
domain_center = np.array([0.5, 0.5])
distances = np.linalg.norm(mesh.cell_centers - domain_center, axis=1)

# Define the water column: radius 0.2, height 1.0m
column_radius = 0.2
column_height = 1.0

# Set the water depth 'h' array directly
dm.h[distances < column_radius] = column_height

# Update the Water Surface Elevation (WSE) after changing h
dm.update_wse()

print("Initial conditions set for column collapse.")

## 4. Running the Simulation and Visualizing

We will run the simulation for a short period and plot the water surface at different times to see the wave propagate.

In [ ]:
def plot_state(model, ax, title):
    """Helper function to create a 2D plot of the water depth."""
    ax.set_aspect('equal')
    # Use tripcolor for unstructured triangular meshes
    tpc = ax.tripcolor(model.mesh.nodes[:, 0], model.mesh.nodes[:, 1], model.mesh.cells,
                      facecolors=model.data_manager.h, cmap='viridis',
                      vmin=0.1, vmax=1.0) # Set color limits
    ax.set_title(title)
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    return tpc

# Create a figure to hold the plots
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Plot Initial State (t=0)
plot_state(model, axes[0], f"Time = {model.current_time:.2f} s")

# --- Simulation Loop ---
plot_times = [0.05, 0.1, 0.2] # Times at which we want to see the state
plot_idx = 1

total_duration = 0.25 # seconds
dt = 0.01 # Main time step
n_steps = int(total_duration / dt)

for i in range(n_steps):
    model.step(dt=dt)
    
    # If the current time is one of our plot times, create a plot
    if plot_idx < len(plot_times) and model.current_time >= plot_times[plot_idx-1]:
        plot_state(model, axes[plot_idx], f"Time = {model.current_time:.2f} s")
        plot_idx += 1

# Add a colorbar
fig.colorbar(axes[3].collections[0], ax=axes.ravel().tolist(), label='Water Depth (m)')

plt.tight_layout()
plt.show()

### Analysis of Results

The plots show the initial column of water at t=0. As time progresses, the column collapses outward, creating a circular wave that propagates across the domain. This demonstrates the model's ability to correctly simulate 2D shallow water flow dynamics.